# Activity — Projection Error

This activity has the following goals:
* Learn how to capture and load an image for analysis
* Learn how to derive a rough estimate of extrinsic and intrinsic parameters by hand
* Learn how to compute and visualize projection error
* Learn how extrinsic and intrinsic parameters are related to projection error

#### Set up notebook

Do all imports.

In [ ]:
# For input/output
from pathlib import Path
import json

# For numerical methods
import numpy as np

# For image processing and visualization of results
import cv2
from pupil_apriltags import Detector
import matplotlib.pyplot as plt

#### Create a tag grid image and template

* "Run all" in the `tags_and_templates.ipynb` example notebook.
* Copy the tag grid image (`.png`) and template (`.json`) generated by the `tags_and_templates.ipynb` notebook into the directory that contains the notebook you're working with now.
* Get a printed copy of the tag grid image from your instructor.
* Define the name of the file that contains your tag grid template in the following cell.

In [ ]:
template_filename = 'name-of-your-template.json' # <-- FIXME

#### Capture one image of the printed tag grid

* Put the printed tag grid on a horizontal, flat surface (e.g., a table or the floor).
* Position your camera directly above the printed tag grid so that the entire tag grid is visible in the image and so that the four edges of the tag grid are roughly parallel to the four edges of the image. Write down the approximate distance in meters between the camera and the tag grid.
* Take a picture with your camera.
* Transfer the picture to your laptop.
* Move the picture into the directory that contains the notebook you're working with now.
* If necessary, convert the picture to `png` format.

Define the name of the image file that contains your picture.

In [ ]:
img_filename = 'name-of-your-image.png' # <-- FIXME

Estimate the position and orientation of the camera with respect to a world frame that is attached to the tag grid (as shown in the `tags_and_templates.ipynb` notebook).

In [ ]:
# Orientation of camera in world frame (FIXME)
R_inW_ofC = np.array([
    [1., 0., 0.],
    [0., 1., 0.],
    [0., 0., 1.],
])

# Position of camera in world frame (FIXME)
p_inW_ofC = np.array([0., 0., 0.])

#### Load and display your image

Use OpenCV to read your image as grayscale (see [list of flags used for image file reading and writing](https://docs.opencv.org/4.x/d8/d6a/group__imgcodecs__flags.html)).

In [ ]:
img = cv2.imread(img_filename, cv2.IMREAD_GRAYSCALE)

Use matplotlib to show your image in this notebook.

In [ ]:
# Create figure
fig, ax = plt.subplots(1, 1)

# Add image to axis
ax.imshow(img, cmap='gray')

# Show figure
plt.show()

Define the width and height of your image in pixels using `img.shape`.

In [ ]:
img_width = 0    # <-- FIXME
img_height = 0   # <-- FIXME

#### Find the upper-left corner of the tag with ID 0 both in the template and the image

Define a function to get a tag with a particular ID from a list of tags.

In [ ]:
def get_tag_with_id(tag_id, tags):
    for tag in tags:
        if tag['tag_id'] == tag_id:
            return tag
    raise Exception(f'tag_id {tag_id} not found in list of tags')

Load the tag grid template.

In [ ]:
with open(template_filename, 'r') as f:
    template = json.load(f)

Get the tag with ID 0 from the tag grid template.

In [ ]:
tag0_template = get_tag_with_id(0, template['tags'])
print(tag0_template)

Get the coordinates (in the world frame) of the **upper left** corner of the tag with ID 0 from the tag grid template.

In [ ]:
p = np.array([0., 0., 0.]) # <-- FIXME

Detect all tags in the image.

In [ ]:
# Create a tag detector
tag_detector = Detector(
    families=template['tag_family'],
    nthreads=1,
    quad_decimate=1.0,
    quad_sigma=0.0,
    refine_edges=1,
    decode_sharpening=0.,
    debug=0,
)

# Apply the tag detector
tag_detections = tag_detector.detect(
    img,
    estimate_tag_pose=False,
    camera_params=None,
    tag_size=None,
)

# Create a list of detected tags
tags = []
for d in tag_detections:
    tags.append({
        'tag_id': d.tag_id,
        'corners': d.corners.tolist(),
    })

Get the tag with ID 0 from the image.

In [ ]:
tag0_image = get_tag_with_id(0, tags)
print(tag0_image)

Get the coordinates (in the image) of the **upper left** corner of the tag with ID 0 from the tag grid image.

In [ ]:
q = np.array([0., 0.]) # <-- FIXME

Show the tag grid image again, this time putting a red dot at the upper left corner of the tag with ID 0 (as detected in the image).

In [ ]:
# Create figure
fig, ax = plt.subplots(1, 1)

# Add image to axis
ax.imshow(img, cmap='gray')

# Put a red dot at the upper left corner of the tag with ID 0
ax.plot(0., 0., 'r.', markersize=12) # <-- FIXME

# Show figure
plt.show()

#### Estimate the instrinic parameters of your camera

Can you think of a way to get a rough estimate these parameters? Some hints:
* For a rough estimate, it is reasonable to assume that the *principal point* (i.e., the point at which the $z$-axis of the camera intersects the image plane) is at the center of the image.
* For a rough estimate, it is reasonable to assume that $f_x = f_y$.
* Remember that you already have an estimate of both the world coordinates and image coordinates of a point.

**FIXME: ADD TEXT HERE TO DESCRIBE YOUR METHOD**

In [ ]:
fx = 0. # <-- FIXME
fy = 0. # <-- FIXME
cx = 0. # <-- FIXME
cy = 0. # <-- FIXME

Define a function to project a point into the image.

In [ ]:
def projection(p_inW, fx, fy, cx, cy, R_inW_ofC, p_inW_ofC):
    # Express world frame in camera frame
    R_inC_ofW = R_inW_ofC.T
    p_inC_ofW = -R_inW_ofC.T @ p_inW_ofC

    # Express point in camera frame
    p_inC = R_inC_ofW @ p_inW + p_inC_ofW

    # Project point
    q = np.array([
        fx * (p_inC[0] / p_inC[2]) + cx,
        fy * (p_inC[1] / p_inC[2]) + cy,
    ])

    # Return result
    return q

Apply this function to project the upper left corner of the tag with ID 0 into the image, using your estimate of extrinsic (i.e., $R_\text{camera}^\text{world}$ and $p_\text{camera}^\text{world}$) and intrinsic (i.e., $f_x$, $f_y$, $c_x$, and $c_y$) parameters of your camera.

In [ ]:
# q_predicted = projection( ... ) # <-- FIXME

Compute the squared reprojection error for the single point we have been considering:

$$\frac{1}{2} \| q - q_\text{predicted} \|^2.$$

This is the quantity that would be minimized (summed over all points in all images) when performing camera calibration.

(You may also want to compute the reprojection error $\| q - q_\text{predicted} \|$ in units of pixels.)

In [ ]:
# error = ... # <-- FIXME

Show the tag grid image again, this time putting a red dot at the upper left corner of the tag with ID 0 (as detected in the image) **and** a blue dot at where you predict this point to be (as given by reprojection with your estimate of intrinsic and extrinsic parameters).

In [ ]:
# Create figure
fig, ax = plt.subplots(1, 1)

# Add image to axis
ax.imshow(img, cmap='gray')

# Put a red dot at the upper left corner of the tag with ID 0 (as detected in the image)
ax.plot(0., 0., 'r.', markersize=12) # <-- FIXME

# Put a red dot at the upper left corner of the tag with ID 0 (as given by reprojection)
ax.plot(0., 0., 'b.', markersize=9) # <-- FIXME

# Show figure
plt.show()

Choose one parameter (either intrinsic or extrinsic) and change your estimate of it in a way that makes the projection error smaller.

**FIXME: ADD TEXT HERE THAT DESCRIBES YOUR METHOD**

In [ ]:
# Change your estimate of one parameter
# ... (FIXME)

# Recompute the projection error to show that it really does get smaller
# ... (FIXME)

# Show the annotated image again to show that the "blue point" really does get closer to the "red point"
# ... (FIXME)